<a href="https://colab.research.google.com/github/eghrthrjdtsed/-------------------------/blob/master/Home_work_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Попробуйте улучшить точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке.

Приложите анализ с описанием того, что улучшает работу нейронной сети, а что ухудшает.

Опишите в анализе, какие изменения необходимо было бы внести в получившуюся нейронную сеть, если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from keras.layers import Dropout
Dropout(rate=0.5)

In [3]:
from keras.layers import BatchNormalization

Sequential([
    BatchNormalization(input_shape=(8, 8, 2))
]).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 8, 8, 2)           8         
 Normalization)                                                  
                                                                 
Total params: 8 (32.00 Byte)
Trainable params: 4 (16.00 Byte)
Non-trainable params: 4 (16.00 Byte)
_________________________________________________________________


In [4]:
# установка параметров нейросети

batch_size = 512
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20
# save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

170498071/170498071 [==============================] - 3s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [5]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [6]:
data_augmentation = True

In [7]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.2,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train,
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 35s 285ms/step - loss: 2.3037 - accuracy: 0.1009 - val_loss: 2.2953 - val_accuracy: 0.1364
Epoch 2/5
98/98 [==============================] - 29s 298ms/step - loss: 2.2923 - accuracy: 0.1195 - val_loss: 2.2835 - val_accuracy: 0.1540
Epoch 3/5
98/98 [==============================] - 27s 277ms/step - loss: 2.2763 - accuracy: 0.1373 - val_loss: 2.2568 - val_accuracy: 0.1904
Epoch 4/5
98/98 [==============================] - 27s 273ms/step - loss: 2.2362 - accuracy: 0.1672 - val_loss: 2.1868 - val_accuracy: 0.2214
Epoch 5/5
98/98 [==============================] - 29s 298ms/step - loss: 2.1604 - accuracy: 0.1967 - val_loss: 2.0826 - val_accuracy: 0.2538


In [8]:
# # сохранение модели и весов
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)

# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
# print(f'сохранить обученную модель как {model_path}')

# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 2.0826 - accuracy: 0.2538
Test loss: 2.082630157470703
Test accuracy: 0.25380000472068787


Точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке, равна 0.2538.


Вношу изменения в Droput:

In [9]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети ответственные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.2,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train,
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [10]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 2.0348 - accuracy: 0.2715
Test loss: 2.034810781478882
Test accuracy: 0.27149999141693115


Распознавания образов cifar 10 сверточной нейронной сетью при изменении Dropout в каждом слое точнее и равна 0.2715.


Вношу изменения в параметры data augmentation:

In [11]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_12 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 15, 15, 32)        0         
                                                      

In [12]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-05,
        rotation_range=5,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.,
        zoom_range=0.15,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train,
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 29s 281ms/step - loss: 2.3063 - accuracy: 0.1053 - val_loss: 2.2966 - val_accuracy: 0.1152
Epoch 2/5
98/98 [==============================] - 29s 293ms/step - loss: 2.2955 - accuracy: 0.1172 - val_loss: 2.2903 - val_accuracy: 0.1338
Epoch 3/5
98/98 [==============================] - 27s 274ms/step - loss: 2.2879 - accuracy: 0.1248 - val_loss: 2.2778 - val_accuracy: 0.1635
Epoch 4/5
98/98 [==============================] - 27s 274ms/step - loss: 2.2714 - accuracy: 0.1419 - val_loss: 2.2531 - val_accuracy: 0.2171
Epoch 5/5
98/98 [==============================] - 26s 269ms/step - loss: 2.2365 - accuracy: 0.1680 - val_loss: 2.1872 - val_accuracy: 0.2300


In [13]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 5ms/step - loss: 2.1872 - accuracy: 0.2300
Test loss: 2.187152147293091
Test accuracy: 0.23000000417232513


Точность распознавания образов cifar 10 сверточной нейронной сетью при изменении некоторых параметров data augmentation улучшело показатели на 4% и составило 0.2300.

Увеличиваю ширину сети (больше фильтров):

In [14]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(64, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 activation_18 (Activation)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 30, 64)        36928     
                                                                 
 activation_19 (Activation)  (None, 30, 30, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 15, 15, 64)        0         
                                                      

In [15]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.2,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train,
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 37s 332ms/step - loss: 2.3020 - accuracy: 0.1044 - val_loss: 2.2978 - val_accuracy: 0.1293
Epoch 2/5
98/98 [==============================] - 27s 280ms/step - loss: 2.2965 - accuracy: 0.1148 - val_loss: 2.2911 - val_accuracy: 0.1405
Epoch 3/5
98/98 [==============================] - 28s 286ms/step - loss: 2.2849 - accuracy: 0.1265 - val_loss: 2.2737 - val_accuracy: 0.1510
Epoch 4/5
98/98 [==============================] - 28s 281ms/step - loss: 2.2621 - accuracy: 0.1441 - val_loss: 2.2329 - val_accuracy: 0.1964
Epoch 5/5
98/98 [==============================] - 28s 285ms/step - loss: 2.2046 - accuracy: 0.1819 - val_loss: 2.1337 - val_accuracy: 0.2450


In [16]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


313/313 [==============================] - 2s 7ms/step - loss: 2.1337 - accuracy: 0.2450
Test loss: 2.133721113204956
Test accuracy: 0.24500000476837158


Точность распознавания образов cifar 10 сверточной нейронной сетью при увеличении ширины сети (больше фильтров) немного повысилась и стала равна 0.2450

Попробую увеличить глубину сети (больше слоев):

In [17]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_24 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_25 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [18]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.2,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train,
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))

Использование data augmentation
Epoch 1/5
98/98 [==============================] - 35s 325ms/step - loss: 2.3035 - accuracy: 0.0988 - val_loss: 2.3023 - val_accuracy: 0.0918
Epoch 2/5
98/98 [==============================] - 29s 298ms/step - loss: 2.3024 - accuracy: 0.1018 - val_loss: 2.3015 - val_accuracy: 0.1447
Epoch 3/5
98/98 [==============================] - 27s 276ms/step - loss: 2.3018 - accuracy: 0.1071 - val_loss: 2.3006 - val_accuracy: 0.1862
Epoch 4/5
98/98 [==============================] - 27s 276ms/step - loss: 2.3006 - accuracy: 0.1162 - val_loss: 2.2994 - val_accuracy: 0.1770
Epoch 5/5
98/98 [==============================] - 27s 274ms/step - loss: 2.2992 - accuracy: 0.1266 - val_loss: 2.2975 - val_accuracy: 0.1766


In [19]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 2.2975 - accuracy: 0.1766
Test loss: 2.29750657081604
Test accuracy: 0.17659999430179596


Точность распознавания образов cifar 10 сверточной нейронной сетью при увеличении глубины сети (больше слоев) была снижена и стала равна 0.1765

Отключаю data augmentation:

In [20]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_32 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_23 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_33 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_17 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [21]:
data_augmentation = False

In [22]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.2,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train,
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit
    model.fit(train_gen,
              epochs=epochs,
              validation_data=(X_test, y_test))


Не используется data augmentation
Epoch 1/5
98/98 [==============================] - 7s 61ms/step - loss: 2.3045 - accuracy: 0.1034 - val_loss: 2.2992 - val_accuracy: 0.1333
Epoch 2/5
98/98 [==============================] - 4s 40ms/step - loss: 2.2980 - accuracy: 0.1157 - val_loss: 2.2940 - val_accuracy: 0.1665
Epoch 3/5
98/98 [==============================] - 4s 42ms/step - loss: 2.2911 - accuracy: 0.1227 - val_loss: 2.2832 - val_accuracy: 0.1614
Epoch 4/5
98/98 [==============================] - 4s 44ms/step - loss: 2.2751 - accuracy: 0.1381 - val_loss: 2.2564 - val_accuracy: 0.1897
Epoch 5/5
98/98 [==============================] - 4s 43ms/step - loss: 2.2296 - accuracy: 0.1707 - val_loss: 2.1737 - val_accuracy: 0.2508


In [23]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 2.1737 - accuracy: 0.2508
Test loss: 2.1737053394317627
Test accuracy: 0.2508000135421753


Точность распознавания образов cifar 10 сверточной нейронной сетью при отключении data augmentation ровняется 0.2508.

## Вывод: </br>
Существенно повысить точность распознования образов cifar 10 сверточной нейронной сетью не удалось показатели точности при разных подходах в диапозоне 0.1765 - 0.2715. Самые худшие показатели были при увеличении глубины сети (больше слоев) = 0.1765, самые лучшие были при изменении Dropout в каждом слое, удалось достичь показатель 0.2715. </br>
Для адаптации полученной нейронной сети для работы с различными датасетами, такими как MNIST, CIFAR100 и IMAGENET, требуется внести следующие изменения: </br>

### MNIST: </br>
Уменьшить глубину сети, так как изображения в MNIST меньше и менее сложны.
Уменьшить число выходных нейронов в последнем слое до 10, соответствующих 10 классам в MNIST.
Изменить размер входного слоя сети на (28, 28, 1), чтобы соответствовать размеру изображений MNIST.
Изменить активационные функции и количество фильтров в сверточных слоях, чтобы лучше соответствовать характеристикам датасета. </br>

### CIFAR100: </br>
Изменить число выходных нейронов в последнем слое до 100, чтобы соответствовать количеству классов в CIFAR100.
Возможно, увеличить глубину сети или количество фильтров в сверточных слоях для более сложных признаков. </br>

### IMAGENET: </br>
Использовать более глубокие и сложные архитектуры сетей, такие как ResNet, Inception или EfficientNet, из-за огромного размера датасета и сложности изображений.
Использовать предобученные модели для эффективного обучения.
Установить соответствующий размер входного слоя сети, обычно (224, 224, 3) для изображений IMAGENET.
Применить более сложные методы аугментации данных, так как датасет IMAGENET гораздо больше по размеру и разнообразию, чем CIFAR-10.